In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 72
import numpy as np
import os
import cv2
import sys
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

import torch
import torchaudio
import torchaudio.transforms as atransforms
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as vtransforms
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import copy

gpu = torch.device('cuda:0')

print(f'PyTorch version= {torch.__version__}')
print(f'torchaudio version= {torchaudio.__version__}')
print(f'torchvision version= {torchvision.__version__}')
print(f'CUDA available= {torch.cuda.is_available()}')


PyTorch version= 2.7.0+cu128
torchaudio version= 2.7.0+cu128
torchvision version= 0.22.0+cu128
CUDA available= True


In [2]:
if torch.cuda.is_available():
    # CUDA Installation
    print('CUDA Version')
    !nvcc --version
    print()

    # CUDNN Installation
    print(f'CUDNN Version: {torch.backends.cudnn.version()}')
    print(f'Number of CUDA Devices: {torch.cuda.device_count()}')
    print(f'Active CUDA Device: {torch.cuda.current_device()}')
    print(f'Available devices: {torch.cuda.device_count()}, Name: {torch.cuda.get_device_name(0)}')
    print(f'Current CUDA device: {torch.cuda.current_device()}')

CUDA Version
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Jan_15_19:38:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.61
Build cuda_12.8.r12.8/compiler.35404655_0

CUDNN Version: 90701
Number of CUDA Devices: 1
Active CUDA Device: 0
Available devices: 1, Name: NVIDIA GeForce RTX 4070
Current CUDA device: 0


In [3]:
def get_dataloader(_img_size, _bs,  _path, val_split=0.1):
    transform = vtransforms.Compose([
        vtransforms.Resize((_img_size, _img_size)),
        vtransforms.ToTensor(),
        vtransforms.Normalize((0.5,), (0.5,))
    ])
    
    full_train_ds = dset.ImageFolder(
        root=os.path.join(_path, 'seg_test', 'seg_test'),
        transform=transform
    )

    # Split into train and validation sets
    val_size = int(len(full_train_ds) * val_split)
    train_size = len(full_train_ds) - val_size
    train_ds, val_ds = random_split(full_train_ds, [train_size, val_size])

    train_dl = DataLoader(train_ds, batch_size=_bs, shuffle=True, pin_memory=True, num_workers=4)
    val_dl   = DataLoader(val_ds,   batch_size=_bs, shuffle=False, pin_memory=True, num_workers=4)

    test_ds = dset.ImageFolder(
        root=os.path.join(_path, 'seg_train', 'seg_train'),
        transform=transform
    )
    test_dl = DataLoader(test_ds, batch_size=_bs, shuffle=False, pin_memory=True, num_workers=4)

    return train_dl, val_dl, test_dl


In [4]:
def evaluate(model, dataloader, loss_func=None, label_names=None, device=None, set_name="Eval", verbose=True):
    model.eval()
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            outputs = model(X_batch)

            if loss_func is not None:
                loss = loss_func(outputs, y_batch)
                total_loss += loss.item() * X_batch.size(0)

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y_batch.cpu().tolist())

    # Accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / len(dataloader.dataset) if loss_func else None

    # Optional verbose metrics
    if verbose:
        print(f"\n--- {set_name} Performance ---")
        print(f"Accuracy: {accuracy * 100:.2f}%")
        if avg_loss is not None:
            print(f"Avg Loss: {avg_loss:.4f}")
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds, target_names=label_names) if label_names else classification_report(all_labels, all_preds))
        print("Confusion Matrix:")
        print(confusion_matrix(all_labels, all_preds))

    return {
        "accuracy": accuracy,
        "loss": avg_loss,
        "preds": all_preds,
        "labels": all_labels
    }

In [5]:
def fit(model, train_dl, lr, epochs, device=gpu, info=False, early_stop=False, patience=3, val_dl=None):
    model.to(device)
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)

    best_loss = float('inf')
    best_model_state = None
    no_improve_counter = 0

    history = {"loss": [], "accuracy": []}

    for e in range(epochs):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        pbar = tqdm(train_dl, desc=f"Epoch {e+1}/{epochs}", leave=False)
        for X, y in pbar:
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            net_out = model(X)
            loss = loss_func(net_out, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X.size(0)
            preds = torch.argmax(net_out, dim=1)
            correct += (preds == y).sum().item()
            total += X.size(0)

            pbar.set_postfix(loss=loss.item())

        avg_loss = total_loss / total
        acc = correct / total
        history["loss"].append(avg_loss)
        history["accuracy"].append(acc)

        if info:
            print(f"Epoch {e+1:02d}/{epochs:02d} | Loss: {avg_loss:.4f} | Acc: {acc*100:.2f}%")

        # Early stopping check
        if early_stop and val_dl is not None:
            val_metrics = evaluate(model, val_dl, loss_func=loss_func, device=device, verbose=False)
            val_loss = val_metrics["loss"]

            if val_loss < best_loss:
                best_loss = val_loss
                best_model_state = model.state_dict()
                no_improve_counter = 0
            else:
                no_improve_counter += 1
                if no_improve_counter >= patience:
                    print(f"\nEarly stopping triggered at epoch {e+1}. Best val loss: {best_loss:.4f}")
                    if best_model_state:
                        model.load_state_dict(best_model_state)
                    break

    return history



In [6]:
def predict(model, X_pred):
    model.eval()
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        if not isinstance(X_pred, torch.Tensor):
            X_pred = torch.FloatTensor(X_pred)
        X_pred = X_pred.to(gpu, non_blocking=True)
        logits = model(X_pred)
        y_pred = torch.argmax(logits, dim=1)
    return y_pred.cpu().numpy()

In [7]:
def build_model(params):

    in_channels = params["in_channels"]
    out_channels = params["out_channels"]
    mid_channel = out_channels * 2
    kernel_size = params["kernel_size"]
    stride = params["stride"]
    padding = params["padding"]
    h_neurons = params["h_neurons"]

    return nn.Sequential(
    #1st Conv Layer
    # Input: _bs*128*128*3
    nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding), # (_bs, 18, 64, 64)
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 18, 32, 32)

    #2nd Conv Layer
    nn.Conv2d(out_channels, mid_channel, kernel_size, stride, padding), #(_bs, 36, 16, 16)
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 36, 8, 8)

    #Flattening for Linear Layer
    nn.Flatten(), #(_bs, 36*8*8=2304)

    #1st FC Layer
    nn.Linear(mid_channel*8*8,  h_neurons), 
    nn.ReLU(inplace=True),

    nn.Dropout(0.5),

    #Final FC Layer
    nn.Linear(h_neurons, 6) # 6 class labels
     
    )

In [8]:
def evaluate_with_seed_variance(params, train_dl, test_dl, label_names=None, num_runs=5, device='cuda'):
    accuracies = []
    all_f1s = []

    for seed in [42 + i for i in range(num_runs)]:
        print(f"\nRun {seed}:")

        # Variable Seed
        torch.manual_seed(seed)
        np.random.seed(seed)

        # New Model
        model = build_model(params).to(device)

        # Train 
        fit(model, train_dl, lr=0.001, epochs=10, info=True)

        # Eval
        metrics = evaluate(model, test_dl, loss_func=torch.nn.CrossEntropyLoss(), label_names=label_names, verbose=False)
        acc = metrics["accuracy"]
        preds = metrics["preds"]
        labels = metrics["labels"]

        accuracies.append(acc)

        # F1-score per class
        _, _, f1, _ = precision_recall_fscore_support(labels, preds, labels=list(range(len(label_names))), zero_division=0)
        all_f1s.append(f1)

    # Summary
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)

    print(f"\nSummary for {num_runs} Runs")
    print(f"Mean Accuracy: {mean_acc * 100:.2f}%")
    print(f"Accuracy Std Dev: {std_acc * 100:.2f}%")

    if label_names:
        all_f1s = np.stack(all_f1s)  
        f1_std = np.std(all_f1s, axis=0)

        print("\nPer-Class F1-Score Std Dev:")
        for cname, std in zip(label_names, f1_std):
            print(f"  {cname}: ±{std:.4f}")


### ^ Helper Functions
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# #1 Create a CNN.  Train on testing portion of dataset.  Report performance.


# CNN Training

In [ ]:
train_dl, val_dl, test_dl = get_dataloader(
    _img_size=128, 
    _bs=128, 
    _path="") ## < Input file path

In [10]:
input_dim = 128 * 128 * 3
in_channels = 3
out_channels = 18
kernel_size = 5
stride = 2
padding = 2


In [11]:
model = nn.Sequential(
    # Input: _bs*128*128*3
    nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding), # (_bs, 18, 64, 64)
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 18, 32, 32)

    nn.Conv2d(out_channels, out_channels*2, kernel_size, stride, padding), #(_bs, 36, 16, 16)
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 36, 8, 8)

    nn.Flatten(), #(_bs, 36*8*8=2304)

    nn.Linear(out_channels*2*8*8,  6) #6 class labels
).to(gpu)

In [12]:
history = fit(model, train_dl, lr=0.001, epochs=10, info=True)

Epoch 01/10 | Loss: 1.4095 | Acc: 45.22%


Epoch 02/10 | Loss: 1.1476 | Acc: 55.22%


Epoch 03/10 | Loss: 0.9758 | Acc: 62.48%


Epoch 04/10 | Loss: 0.9006 | Acc: 65.93%


Epoch 05/10 | Loss: 0.9171 | Acc: 64.70%


Epoch 06/10 | Loss: 0.8330 | Acc: 69.11%


Epoch 07/10 | Loss: 0.7618 | Acc: 72.37%


Epoch 08/10 | Loss: 0.7312 | Acc: 73.07%


Epoch 09/10 | Loss: 0.7235 | Acc: 74.07%


Epoch 10/10 | Loss: 0.6711 | Acc: 76.00%


# Evaluate Model

In [13]:
loss_func = nn.CrossEntropyLoss()
label_names = test_dl.dataset.classes

train_metrics = evaluate(model, train_dl, loss_func=loss_func, label_names=label_names, set_name="Train")

test_metrics = evaluate(model, test_dl, loss_func=loss_func, label_names=label_names, set_name="Test")


--- Train Performance ---
Accuracy: 78.67%
Avg Loss: 0.6132

Classification Report:
              precision    recall  f1-score   support

   buildings       0.74      0.75      0.75       381
      forest       0.86      0.95      0.90       429
     glacier       0.76      0.76      0.76       500
    mountain       0.74      0.76      0.75       476
         sea       0.74      0.70      0.72       460
      street       0.88      0.81      0.84       454

    accuracy                           0.79      2700
   macro avg       0.79      0.79      0.79      2700
weighted avg       0.79      0.79      0.79      2700

Confusion Matrix:
[[285  23  15  19   9  30]
 [ 11 406   1   2   0   9]
 [ 12   3 380  45  54   6]
 [ 16   3  43 364  48   2]
 [ 22   5  51  56 322   4]
 [ 38  31   9   6   3 367]]

--- Test Performance ---
Accuracy: 70.60%
Avg Loss: 0.8118

Classification Report:
              precision    recall  f1-score   support

   buildings       0.66      0.64      0.65      219

# #2: Add in regularization and/or drop-out

In [14]:
train_dl, val_dl, test_dl = get_dataloader(
    _img_size=128, 
    _bs=128, 
    _path="C:/Users/nelan/EN705_ML/Datasets/Mod13")

input_dim = 128 * 128 * 3
in_channels = 3
out_channels = 18
kernel_size = 5
stride = 2
padding = 2
h_neurons = 512

In [15]:
model = nn.Sequential(
    #1st Conv Layer
    # Input: _bs*128*128*3
    nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding), # (_bs, 18, 64, 64)
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 18, 32, 32)

    #2nd Conv Layer
    nn.Conv2d(out_channels, out_channels*2, kernel_size, stride, padding), #(_bs, 36, 16, 16)
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 36, 8, 8)

    #Flattening for Linear Layer
    nn.Flatten(), #(_bs, 36*8*8=2304)

    #1st FC Layer
    nn.Linear(out_channels*2*8*8,  h_neurons), 
    nn.ReLU(inplace=True),

    nn.Dropout(0.5),

    #Final FC Layer
    nn.Linear(h_neurons, 6) # 6 class labels
     
).to(gpu)

In [16]:
history = fit(model, train_dl, lr=0.001, epochs=10, info=True, early_stop=True, val_dl=val_dl)

Epoch 01/10 | Loss: 1.4426 | Acc: 41.52%


Epoch 02/10 | Loss: 1.1529 | Acc: 54.67%


Epoch 03/10 | Loss: 0.9797 | Acc: 61.11%


Epoch 04/10 | Loss: 0.8914 | Acc: 65.22%


Epoch 05/10 | Loss: 0.8942 | Acc: 66.33%


Epoch 06/10 | Loss: 0.8175 | Acc: 69.37%


Epoch 07/10 | Loss: 0.7607 | Acc: 71.11%


Epoch 08/10 | Loss: 0.7091 | Acc: 73.96%


Epoch 09/10 | Loss: 0.7055 | Acc: 73.41%


Epoch 10/10 | Loss: 0.6439 | Acc: 76.30%


In [17]:
loss_func = nn.CrossEntropyLoss()
label_names = test_dl.dataset.classes

train_metrics = evaluate(model, train_dl, loss_func=loss_func, label_names=label_names, set_name="Train")

test_metrics = evaluate(model, test_dl, loss_func=loss_func, label_names=label_names, set_name="Test")


--- Train Performance ---
Accuracy: 80.89%
Avg Loss: 0.5455

Classification Report:
              precision    recall  f1-score   support

   buildings       0.80      0.79      0.80       393
      forest       0.93      0.92      0.93       427
     glacier       0.81      0.76      0.79       492
    mountain       0.79      0.69      0.74       475
         sea       0.68      0.84      0.75       463
      street       0.88      0.86      0.87       450

    accuracy                           0.81      2700
   macro avg       0.82      0.81      0.81      2700
weighted avg       0.81      0.81      0.81      2700

Confusion Matrix:
[[311  12   9  15  18  28]
 [  9 393   1   2   2  20]
 [  7   2 375  34  70   4]
 [ 10   1  44 329  91   0]
 [ 12   0  27  33 391   0]
 [ 38  13   5   5   4 385]]

--- Test Performance ---
Accuracy: 71.85%
Avg Loss: 0.7718

Classification Report:
              precision    recall  f1-score   support

   buildings       0.69      0.67      0.68      219

### Check performance standard deviation and comment on its indication of robustness

In [18]:
params = {
    'input_dim' : 128 * 128 * 3,
    'in_channels' : 3,
    'out_channels' : 18,
    'kernel_size' : 5,
    'stride' : 2,
    'padding' : 2,
    'h_neurons' : 512
}

evaluate_with_seed_variance(params, train_dl, test_dl, label_names)


Run 42:


Epoch 01/10 | Loss: 1.4252 | Acc: 43.67%


Epoch 02/10 | Loss: 1.0879 | Acc: 57.93%


Epoch 03/10 | Loss: 0.9591 | Acc: 61.78%


Epoch 04/10 | Loss: 0.9284 | Acc: 62.44%


...

Epoch 06/10 | Loss: 0.8159 | Acc: 69.41%


Epoch 07/10 | Loss: 0.7485 | Acc: 73.22%


Epoch 08/10 | Loss: 0.7165 | Acc: 74.22%


Epoch 09/10 | Loss: 0.7399 | Acc: 72.52%


Epoch 10/10 | Loss: 0.6520 | Acc: 75.93%

Summary for 5 Runs
Mean Accuracy: 70.30%
Accuracy Std Dev: 0.38%

Per-Class F1-Score Std Dev:
  buildings: ±0.0233
  forest: ±0.0239
  glacier: ±0.0175
  mountain: ±0.0076
  sea: ±0.0120
  street: ±0.0305


### Commentary:  The low performance standard deviation is indicative of a higher global robustness.  If the standard deviation was higher, it would be indicative of the model "accidentally" performing well due to initialization or data order.  The low "per-class" f1 score std deviation suggests that each class is being learned consistently across runs without class-specific instability.

# #3 Add batch normalization and early stopping.

In [19]:
train_dl, val_dl, test_dl = get_dataloader(
    _img_size=128, 
    _bs=128, 
    _path="C:/Users/nelan/EN705_ML/Datasets/Mod13")

input_dim = 128 * 128 * 3
in_channels = 3
out_channels = 18
kernel_size = 5
stride = 2
padding = 2
h_neurons = 512

In [20]:
model = nn.Sequential(
    #1st Conv Layer
    # Input: _bs*128*128*3
    nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding), # (_bs, 18, 64, 64)
    nn.BatchNorm2d(out_channels),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 18, 32, 32)


    #2nd Conv Layer
    nn.Conv2d(out_channels, out_channels*2, kernel_size, stride, padding), #(_bs, 36, 16, 16)
    nn.BatchNorm2d(out_channels*2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2), # (_bs, 36, 8, 8)

    #Flattening for Linear Layer
    nn.Flatten(), #(_bs, 36*8*8=2304)

    #1st FC Layer
    nn.Linear(out_channels*2*8*8,  h_neurons), 
    nn.ReLU(inplace=True),

    nn.Dropout(0.5),

    #Final FC Layer
    nn.Linear(h_neurons, 6) # 6 class labels
     
).to(gpu)

In [21]:
history = fit(model, train_dl, lr=0.001, epochs=10, info=True, early_stop=True, val_dl=val_dl)

loss_func = nn.CrossEntropyLoss()
label_names = test_dl.dataset.classes

train_metrics = evaluate(model, train_dl, loss_func=loss_func, label_names=label_names, set_name="Train")

test_metrics = evaluate(model, test_dl, loss_func=loss_func, label_names=label_names, set_name="Test")

Epoch 01/10 | Loss: 1.4163 | Acc: 45.74%


Epoch 02/10 | Loss: 1.0354 | Acc: 61.22%


Epoch 03/10 | Loss: 0.9133 | Acc: 64.93%


Epoch 04/10 | Loss: 0.8455 | Acc: 68.00%


Epoch 05/10 | Loss: 0.7606 | Acc: 71.04%


Epoch 06/10 | Loss: 0.7520 | Acc: 71.37%


Epoch 07/10 | Loss: 0.6935 | Acc: 73.70%


Epoch 08/10 | Loss: 0.6716 | Acc: 74.93%


Epoch 09/10 | Loss: 0.6251 | Acc: 76.56%


Epoch 10/10 | Loss: 0.6325 | Acc: 76.33%

--- Train Performance ---
Accuracy: 82.07%
Avg Loss: 0.5006

Classification Report:
              precision    recall  f1-score   support

   buildings       0.81      0.80      0.80       389
      forest       0.92      0.92      0.92       424
     glacier       0.87      0.72      0.79       501
    mountain       0.87      0.70      0.78       480
         sea       0.68      0.92      0.78       462
      street       0.84      0.89      0.86       444

    accuracy                           0.82      2700
   macro avg       0.83      0.82      0.82      2700
weighted avg       0.83      0.82      0.82      2700

Confusion Matrix:
[[311  15   0   4  18  41]
 [  4 390   0   3   1  26]
 [ 10   2 361  30  93   5]
 [ 13   4  44 337  81   1]
 [ 16   2   8  12 424   0]
 [ 31  11   0   1   8 393]]

--- Test Performance ---
Accuracy: 73.27%
Avg Loss: 0.7111

Classification Report:
              precision    recall  f1-score   support

   building

### Commentary on Effectiveness:

- Training Accuracy improved by 4%  indicating that the model was able to optimize more effectively while maintaining stability.  This suggests that the BatchNorm helped stabilize internal activations allowing for deeper/faster learning across epochs.

- Test Accuracy improved by 3% indicating the combined effect of regularization preventing the model from overfitting past the point of max validation performance.

- Overall the additions resulted in a more robust and stable model design.  BatchNorm made the model less sensitive to initialization and data scaling.  Early Stop ensured the model retained generalization capacity without overfitting.